In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Transform").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/18 18:31:16 WARN Utils: Your hostname, Kavanas-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.11 instead (on interface en0)
25/11/18 18:31:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/18 18:31:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/18 18:31:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/18 18:31:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [37]:
reviews_df = spark.read.parquet("/Users/kavanamanvi/Desktop/AmazonReviews/processed/bronze/reviews_raw")


25/11/19 01:57:23 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: /Users/kavanamanvi/Desktop/AmazonReviews/processed/bronze/reviews_raw.
java.io.FileNotFoundException: File /Users/kavanamanvi/Desktop/AmazonReviews/processed/bronze/reviews_raw does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loa

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/kavanamanvi/Desktop/AmazonReviews/processed/bronze/reviews_raw. SQLSTATE: 42K03

In [7]:
reviews_df.show(5)

+----------+------------+------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+
|      asin|helpful_vote|images|parent_asin|rating|                text|    timestamp|               title|             user_id|verified_purchase|
+----------+------------+------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+
|B004H0J5QY|           0|    []| B004HD55V0|   5.0|Great game for th...|1398016704000|Great game for th...|AHP4ABT4AOUOKHKXC...|             true|
|B00AR5BNYU|           1|    []| B00AR5BNYU|   5.0|Great graphics,no...|1370048921000|Great value,for t...|AHP4ABT4AOUOKHKXC...|            false|
|B003MQMPD4|           4|    []| B003NE6BQW|   1.0|I wish I could gi...|1326939063000|Do not waste the ...|AHP4ABT4AOUOKHKXC...|             true|
|B003L77ZHK|           3|    []| B003L77ZHK|   4.0|I have been looki...|1326502126000|If looking for a ...|AHP4ABT4AOU

In [9]:
reviews_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful_vote: long (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attachment_type: string (nullable = true)
 |    |    |-- large_image_url: string (nullable = true)
 |    |    |-- medium_image_url: string (nullable = true)
 |    |    |-- small_image_url: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- verified_purchase: boolean (nullable = true)



## 2B) Clean Data

In [12]:
#selecting only the columns I will need for Recc algo
clean_df = reviews_df.select("user_id", "asin", "rating", "timestamp")

In [14]:
clean_df.show(5)
clean_df.printSchema()

+--------------------+----------+------+-------------+
|             user_id|      asin|rating|    timestamp|
+--------------------+----------+------+-------------+
|AHP4ABT4AOUOKHKXC...|B004H0J5QY|   5.0|1398016704000|
|AHP4ABT4AOUOKHKXC...|B00AR5BNYU|   5.0|1370048921000|
|AHP4ABT4AOUOKHKXC...|B003MQMPD4|   1.0|1326939063000|
|AHP4ABT4AOUOKHKXC...|B003L77ZHK|   4.0|1326502126000|
|AEHINTI4PJFF3EQ7O...|B07TL5TKG9|   5.0|1612121491449|
+--------------------+----------+------+-------------+
only showing top 5 rows
root
 |-- user_id: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)



### Step 2B.2)Remove rows with missing key values

Why?

Because:

user_id cannot be null

asin cannot be null

rating cannot be null

In [17]:
clean_df = clean_df.dropna(subset=["user_id", "asin", "rating"])

In [19]:
clean_df.show(5)

+--------------------+----------+------+-------------+
|             user_id|      asin|rating|    timestamp|
+--------------------+----------+------+-------------+
|AHP4ABT4AOUOKHKXC...|B004H0J5QY|   5.0|1398016704000|
|AHP4ABT4AOUOKHKXC...|B00AR5BNYU|   5.0|1370048921000|
|AHP4ABT4AOUOKHKXC...|B003MQMPD4|   1.0|1326939063000|
|AHP4ABT4AOUOKHKXC...|B003L77ZHK|   4.0|1326502126000|
|AEHINTI4PJFF3EQ7O...|B07TL5TKG9|   5.0|1612121491449|
+--------------------+----------+------+-------------+
only showing top 5 rows


### Step 2B.3) Convert timestamp into real date

In [23]:
from pyspark.sql.functions import from_unixtime, to_timestamp

clean_df = clean_df.withColumn(
    "event_time",
    to_timestamp(from_unixtime("timestamp"))
)

In [25]:
clean_df.show(5)

+--------------------+----------+------+-------------+--------------------+
|             user_id|      asin|rating|    timestamp|          event_time|
+--------------------+----------+------+-------------+--------------------+
|AHP4ABT4AOUOKHKXC...|B004H0J5QY|   5.0|1398016704000|+46271-05-21 16:2...|
|AHP4ABT4AOUOKHKXC...|B00AR5BNYU|   5.0|1370048921000|+45385-02-13 10:4...|
|AHP4ABT4AOUOKHKXC...|B003MQMPD4|   1.0|1326939063000|+44019-01-09 18:1...|
|AHP4ABT4AOUOKHKXC...|B003L77ZHK|   4.0|1326502126000|+44005-03-06 14:4...|
|AEHINTI4PJFF3EQ7O...|B07TL5TKG9|   5.0|1612121491449|+53056-02-05 07:2...|
+--------------------+----------+------+-------------+--------------------+
only showing top 5 rows


### Step 2B.4) Inspect the cleaned dataset

In [30]:
clean_df.printSchema()
clean_df.show(10)
clean_df.count()

root
 |-- user_id: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- event_time: timestamp (nullable = true)

+--------------------+----------+------+-------------+--------------------+
|             user_id|      asin|rating|    timestamp|          event_time|
+--------------------+----------+------+-------------+--------------------+
|AHP4ABT4AOUOKHKXC...|B004H0J5QY|   5.0|1398016704000|+46271-05-21 16:2...|
|AHP4ABT4AOUOKHKXC...|B00AR5BNYU|   5.0|1370048921000|+45385-02-13 10:4...|
|AHP4ABT4AOUOKHKXC...|B003MQMPD4|   1.0|1326939063000|+44019-01-09 18:1...|
|AHP4ABT4AOUOKHKXC...|B003L77ZHK|   4.0|1326502126000|+44005-03-06 14:4...|
|AEHINTI4PJFF3EQ7O...|B07TL5TKG9|   5.0|1612121491449|+53056-02-05 07:2...|
|AFVOXOECS5WI3ZWDO...|B00BT9DTC2|   5.0|1464131745000|+48366-06-28 01:3...|
|AEI5JLLF4OUBDQSBF...|B00K586O7K|   1.0|1410276567000|+46659-11-20 04:5...|
|AFR75TQBIL7AJJV3O...|B004YCRITQ|   5

4624615

### Step 2B.5) Save CLEANED data to Parquet

In [35]:
clean_df.write.mode("overwrite").parquet("/Users/kavanamanvi/Desktop/AmazonReviews/processed/silver/reviews_clean")